Reference

- [AudioSet](https://research.google.com/audioset/)

More model ideas

- Other (smaller) versions of AST

- https://huggingface.co/topel/ConvNeXt-Tiny-AT

- https://huggingface.co/search/full-text?q=audioset&p=1&type=model

- https://paperswithcode.com/paper/efficient-large-scale-audio-tagging-via

- https://paperswithcode.com/paper/dynamic-convolutional-neural-networks-as

- https://paperswithcode.com/paper/panns-large-scale-pretrained-audio-neural-1

# Imports, installs, etc.

In [ ]:
!pip install -qq transformers

In [ ]:
import requests
import sys
import time
import json

import numpy as np

from tqdm.notebook import tqdm

import torch
import torchaudio

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In [ ]:
sys.path.append('./drive/MyDrive/Projects/MiniSoundFinder_v2/experiments/library/')

import events, preprocess

In [ ]:
# from importlib import reload
# reload(event_finder)

# Samples

In [ ]:
!cp ./drive/MyDrive/Projects/MiniSoundFinder_v2/samples/* .

In [ ]:
sample_path = 'freesound_442485_dogs_barking_60sec.wav'
print(torchaudio.info(sample_path))

In [ ]:
waveform, sampling_rate = torchaudio.load(sample_path)
waveform.shape

In [ ]:
wf_prep = preprocess.convert_audio(waveform, sampling_rate)
wf_prep.shape

# Models

## AST

- [AST on HuggingFace](https://huggingface.co/MIT/ast-finetuned-audioset-10-10-0.4593)

- [AST Paper](https://arxiv.org/pdf/2104.01778.pdf)

In [ ]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification

In [ ]:
extractor_ast = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
extractor_ast

In [ ]:
model_ast = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593").to(DEVICE)
model_ast

In [ ]:
target_sampling_rate = extractor_ast.sampling_rate
wf_prep = preprocess.convert_audio(waveform, sampling_rate, sampling_rate=target_sampling_rate)
sample_features = extractor_ast(wf_prep, target_sampling_rate, return_tensors="pt").to(DEVICE)
sample_features['input_values'].shape

In [ ]:
with torch.no_grad():
    probs = torch.sigmoid(model_ast(**sample_features).logits)

In [ ]:
top_classes = torch.argsort(probs, dim=-1, descending=True).flatten()[:10]
top_labels = [(model_ast.config.id2label[id.item()], probs[0, id].item()) for id in top_classes]
top_labels

In [ ]:
def measure_inference_time_ast(model, feature_extractor,
                               sample_length_sec=60,
                               repeats=10,
                               chunk_length_sec=10):

    sampling_rate = feature_extractor.sampling_rate
    sample_length = sampling_rate * sample_length_sec

    extr_times = []
    inf_times = []
    for i in tqdm(range(repeats)):
        extr_start = time.time()
        wf = torch.distributions.uniform.Uniform(-10000, 10000).sample((sample_length,))
        chunks = event_finder.chunk_audio(wf, sampling_rate, chunk_length_sec)
        inp = feature_extractor(chunks, sampling_rate, return_tensors="pt").to(DEVICE)
        extr_times.append(time.time() - extr_start)

        inf_start = time.time()
        with torch.no_grad():
            probs = torch.sigmoid(model(**inp).logits)
        inf_times.append(time.time() - inf_start)

    print("Extraction:", np.mean(extr_times), "±", np.std(extr_times))
    print("Inference:", np.mean(inf_times), "±", np.std(inf_times))

# print("1 minute")
# measure_inference_time_ast(model_ast, extractor_ast, sample_length_sec=60)
# print()

# print("2 minutes")
# measure_inference_time_ast(model_ast, extractor_ast, sample_length_sec=120)
# print()

# print("5 minutes")
# measure_inference_time_ast(model_ast, extractor_ast, sample_length_sec=300)
# print()

## AST Distilled

https://huggingface.co/bookbot/distil-ast-audioset

In [ ]:
extractor_ast_distil = AutoFeatureExtractor.from_pretrained("bookbot/distil-ast-audioset")
extractor_ast_distil

In [ ]:
model_ast_distil = ASTForAudioClassification.from_pretrained("bookbot/distil-ast-audioset").to(DEVICE)
model_ast_distil

In [ ]:
target_sampling_rate = extractor_ast_distil.sampling_rate
wf_prep = preprocess.convert_audio(waveform, sampling_rate, sampling_rate=target_sampling_rate)
sample_features = extractor_ast_distil(wf_prep, target_sampling_rate, return_tensors="pt").to(DEVICE)

with torch.no_grad():
    probs = torch.sigmoid(model_ast_distil(**sample_features).logits)

top_classes = torch.argsort(probs, dim=-1, descending=True).flatten()[:10]
top_labels = [(model_ast.config.id2label[id.item()], probs[0, id].item()) for id in top_classes]
top_labels

In [ ]:
# print("1 minute")
# measure_inference_time_ast(model_ast_distil, extractor_ast_distil, sample_length_sec=60)
# print()

# print("2 minutes")
# measure_inference_time_ast(model_ast_distil, extractor_ast_distil, sample_length_sec=120)
# print()

# print("5 minutes")
# measure_inference_time_ast(model_ast_distil, extractor_ast_distil, sample_length_sec=300)
# print()

# Finding events

In [ ]:
!cp ./drive/MyDrive/Projects/MiniSoundFinder_v2/experiments/ontology.json .

In [ ]:
with open("ontology.json", "r") as f:
    nodes = json.load(f)

In [ ]:
# NOTE: library usage here is outdated

model = model_ast
extractor = extractor_ast
chunk_length_sec = 10
sampling_rate = extractor.sampling_rate

finder = events.EventFinder(
    model.config.id2label,
    nodes,
    chunk_length_sec=chunk_length_sec,
    probability_threshold=0.2)

def find_events(audio_path):
    waveform_raw, source_sampling_rate = torchaudio.load(audio_path)
    waveform = preprocess.convert_audio(waveform_raw, source_sampling_rate,
                                        channels="mono", sampling_rate=sampling_rate)
    chunks = preprocess.chunk_audio(waveform, sampling_rate, chunk_length_sec=chunk_length_sec)

    features = extractor(chunks, sampling_rate, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        probs = torch.sigmoid(model(**features).logits)

    events = finder.find(probs)
    return probs, events

In [ ]:
probs, events = find_events('/content/freesound_442485_dogs_barking_60sec.wav')
events

In [ ]:
probs, events = find_events('/content/freesound_471408_birds_90sec.wav')
events

In [ ]:
probs, events = find_events('/content/recorded_street_150sec.wav')
events

In [ ]:
# probs, top_labels = find_events('/content/freesound_442485_dogs_barking_60sec.wav')
# top_classes = torch.argsort(probs, dim=-1, descending=True)[:, :5]
# for i in range(top_classes.shape[0]):
#     print([(model.config.id2label[id.item()], id.item(), probs[i, id].item()) for id in top_classes[i]])